In [23]:
import warnings
warnings.filterwarnings("ignore")

import torch
import torch.nn as nn
import librosa

import soundfile as sf
import fairseq

class Dict2Obj(object):
    def __init__(self, dictionary):
        """Constructor"""
        for key in dictionary:
            setattr(self, key, dictionary[key])

In [36]:
fs = 16000
# ckpt_path = "/mntcephfs/lab_data/shoinoue/Models/trained_models/wav2vec/w2v_large_lv_fsh_swbd_cv_ftls960_updated.pt"
ckpt_path = "/mntcephfs/lab_data/shoinoue/Models/trained_models/wav2vec/w2v_large_lv_fsh_swbd_cv_ftls960_updated_new.pt"
path = "/mntcephfs/lab_data/shoinoue/Dataset/L2-ARCTIC/HKK/wav/arctic_a0001.wav"

state = torch.load(ckpt_path)
# state["args"] = state["cfg"]
# state["args"] = Dict2Obj(state["args"])
# torch.save(state, ckpt_path[:-3] + "_new.pt")

In [37]:
from fairseq import tasks

In [39]:
tasks.setup_task(state["cfg"])

AttributeError: 'dict' object has no attribute 'task'

In [ ]:
state["args"].task.build_model()

In [33]:
def read_audio(fname, fs=16000):
    """ Load an audio file and return PCM along with the sample rate """

    wav, sr = sf.read(fname)
    if sr!=fs:
        wav, _ = librosa.load(fname, fs)
        sf.write("temp.wav", wav, fs, subtype="PCM_24") # 書き込み
        wav, sr = sf.read("temp.wav")
    assert sr == fs

    return wav, fs

In [34]:
class PretrainedWav2VecModel(nn.Module):
    def __init__(self, fname):
        super().__init__()

        model, cfg, task = fairseq.checkpoint_utils.load_model_ensemble_and_task([fname])
        model = model[0]
        model.eval()

        self.model = model

    def forward(self, x):
        with torch.no_grad():
            z = self.model.feature_extractor(x)
            if isinstance(z, tuple):
                z = z[0]
            c = self.model.feature_aggregator(z)
        return z, c
    
class Prediction:
    """ Lightweight wrapper around a fairspeech embedding model """

    def __init__(self, fname, gpu=0):
        self.gpu = gpu
        self.model = PretrainedWav2VecModel(fname).cuda(gpu)

    def __call__(self, x):
        x = torch.from_numpy(x).float().cuda(self.gpu)
        with torch.no_grad():
            z, c = self.model(x.unsqueeze(0))

        return z.squeeze(0).cpu().numpy(), c.squeeze(0).cpu().numpy()

In [35]:
model = Prediction(ckpt_path, 0)
wav, sr = read_audio(path)

TypeError: unhashable type: 'dict'

In [ ]:
z, c = model(wav)